In [1]:
import os
import json
import csv
import pandas as pd
from src import string_matching, features

In [2]:
ELEMENTS = {
    'com': 'community',
    'env': 'environment',
    'div': 'diversity',
    'emp': 'employee_relation',
    'pro': 'product',
}

In [3]:
dffortune = pd.read_csv('../data/fortune/f500_full_firm_data.csv')
df_mscikld = pd.read_csv('../data/dbs/msci_esg_kld.csv', low_memory=False)

dffortune_sample = dffortune.loc[lambda x: (x['ranking']<=300) & (x['include']==True)].drop(columns=['include', 'ranklabel']).reset_index(drop=True)

df_mscikld = df_mscikld.rename(columns={'CompanyName': 'firm'})
df_mscikld, _, __ = string_matching.match_firm_hash(dffortune_sample, df_mscikld, verbose=False)
df_mscikld_matched = df_mscikld.loc[lambda x: x['firmhash']>0]

In [4]:
csr_scores = features.generate_csr_index(df_mscikld_matched, ELEMENTS)

In [5]:
with open('../data/msci_kld/msci_kld_social_ratings.json', 'w') as outfile:
    json.dump(csr_scores, outfile)

## to pandas

In [6]:
with open('../data/msci_kld/msci_kld_social_ratings.json', 'r') as infile:
    csr_scores = json.load(infile)

In [7]:
df_csr_scores = pd.DataFrame({
    'firmhash': [k for k in csr_scores.keys()],
    'last_available_year': [v['last_available_year'] for v in csr_scores.values()],
    'csr_index': [round(v['csr_index'], 10) for v in csr_scores.values()]
})

In [8]:
df_csr_scores.to_csv('../data/msci_kld/msci_kld_social_ratings.csv',
                     index=False,
                     quotechar='"',
                     quoting=csv.QUOTE_NONNUMERIC)